In [1]:
import jpy_tools.parseSnake2 as jps

In [2]:
path_config = "/public/home/liuzj/scripts/pipeline/basecallByGuppy/snakemake/config.yaml"
path_sf = "/public/home/liuzj/scripts/pipeline/basecallByGuppy/snakemake/snakefile"
dir_scripts = "/scem/work/liuzj/github/Liuzj_allScripts/pipeline/basecallByGuppy/scripts/"

In [3]:
sf = jps.SnakeFile()

In [4]:
header = jps.SnakeHeader(sf, path_config)
header

/public/home/liuzj/softwares/anaconda3/lib/python3.8/site-packages/jpy_tools/parseSnake2.py:93: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  self.yaml = yaml.load(open(self.path))


import pandas as pd
#configfile: "/public/home/liuzj/scripts/pipeline/basecallByGuppy/snakemake/config.yaml"
pipelineDir = config['pipelineDir']
resultDir = config["resultDir"].rstrip("/") + "/"
pipelineDir = config["pipelineDir"].rstrip("/") + "/"

In [5]:
config = header.getConfig()
resultDir = config["resultDir"].rstrip("/") + "/"
pipelineDir = config["pipelineDir"].rstrip("/") + "/"

In [6]:
config

{'pipelineDir': '/scem/work/liuzj/github/Liuzj_allScripts/pipeline/basecallByGuppy/scripts/',
 'resultDir': '/scem/work/liuzj/projects/mouse/basecalledFastq/',
 'guppy': '~/softwares/ont-guppy-4.2/bin/guppy_basecaller',
 'model': '~/softwares/ont-guppy-4.2/data/dna_r9.4.1_450bps_hac.cfg',
 'input': {'all': {'path': '/scem/work/liuzj/projects/mouse/allFast5/20210723_1345_MN29338_FAQ41752_7386409e/fast5/',
   'nparts': 12,
   'need_h5': False}}}

In [7]:
import pandas as pd
dt_h5 = config['input']
df_splitH5 = pd.DataFrame.from_dict(dt_h5).T
df_splitH5['dir_output'] = df_splitH5.index + '/'

In [8]:
rule_splitH5 = jps.SnakeRule(sf, 'splitH5', 1)
rule_splitH5.addCode("""
import pandas as pd
dt_h5 = config['input']
df_splitH5 = pd.DataFrame.from_dict(dt_h5).T
df_splitH5['dir_output'] = df_splitH5.index + '/'
""")
rule_splitH5.addMetaDf('df_splitH5', ['dir_output'], df_splitH5)
rule_splitH5.addMain('input', ['path'])
rule_splitH5.addMain('params', ['nparts', 'dir_output'])
rule_splitH5.setShell("""
cd {pipelineDir}
python ./splitFast5ToMultipleDir.py -i {input.path} -o {params.dir_output} -n {params.nparts}
""")
rule_splitH5

2021-09-27 15:17:45.740 | INFO     | jpy_tools.parseSnake2:addRule:55 - splitH5 step num: 1



## get parameter of rule `splitH5` ##
import pandas as pd
dt_h5 = config['input']
df_splitH5 = pd.DataFrame.from_dict(dt_h5).T
df_splitH5['dir_output'] = df_splitH5.index + '/'
for column in ['dir_output']:
    df_splitH5[column] = resultDir + 'step1_splitH5/' + df_splitH5[column]
----------------
IN RULE
----------------
# parameter's dataframe of splitH5: 
# |     | need_h5   |   nparts | path                                                                                    | dir_output   |
# |:----|:----------|---------:|:----------------------------------------------------------------------------------------|:-------------|
# | all | False     |       12 | /scem/work/liuzj/projects/mouse/allFast5/20210723_1345_MN29338_FAQ41752_7386409e/fast5/ | all/         |
rule splitH5:
    input:
        path = lambda wildcard: df_splitH5.at[wildcard.sample, 'path'],
    output:
        splitH5Finished = resultDir + 'step1_splitH5/' + '{sample}.finished',
    params:
        gpu = 0,
        

In [9]:
df_basecall = df_splitH5.copy()
df_basecall["nparts"] = df_basecall["nparts"].map(lambda x: list(range(x)))
df_basecall = df_basecall.explode("nparts")
df_basecall = df_basecall[["nparts", "dir_output"]]
df_basecall['sample'] = df_basecall.index
df_basecall.index = df_basecall.index + df_basecall.nparts.astype(str)
df_basecall.dir_output = df_basecall.dir_output + df_basecall.nparts.astype(str) + '/'
df_basecall['basecalledDir'] = df_basecall.index + '/'
df_basecall['guppy'] = config['guppy']
df_basecall['model'] = config['model']

In [10]:
rule_basecall = jps.SnakeRule(sf, "basecall", 18, 2, wildCard="sampleSplit")
rule_basecall.addCode(
    """
df_basecall = df_splitH5.copy()
df_basecall["nparts"] = df_basecall["nparts"].map(lambda x: list(range(x)))
df_basecall = df_basecall.explode("nparts")
df_basecall = df_basecall[["nparts", "dir_output"]]
df_basecall['sample'] = df_basecall.index
df_basecall.index = df_basecall.index + df_basecall.nparts.astype(str)
df_basecall.dir_output = df_basecall.dir_output + df_basecall.nparts.astype(str) + '/'
df_basecall['basecalledDir'] = df_basecall.index + '/'
df_basecall['guppy'] = config['guppy']
df_basecall['model'] = config['model']
"""
)
rule_basecall.addMetaDf("df_basecall", ["basecalledDir"], metaDf = df_basecall)
rule_basecall.addMain("params", ["need_h5"], rule_splitH5)
rule_basecall.addMain("params", ["dir_output", "basecalledDir", "guppy", "model"])

rule_basecall.setShell("""
if [ {params.need_h5} = True ]
then
    {params.guppy} -c {params.model} -i {params.dir_output} --qscore_filtering --min_qscore=7 -s {params.basecalledDir} -x "cuda:all:100%" --disable_pings --fast5_out
else
    {params.guppy} -c {params.model} -i {params.dir_output} --qscore_filtering --min_qscore=7 -s {params.basecalledDir} -x "cuda:all:100%" --disable_pings
fi
""")
rule_basecall

2021-09-27 15:18:27.380 | INFO     | jpy_tools.parseSnake2:addRule:55 - basecall step num: 2



## get parameter of rule `basecall` ##
df_basecall = df_splitH5.copy()
df_basecall["nparts"] = df_basecall["nparts"].map(lambda x: list(range(x)))
df_basecall = df_basecall.explode("nparts")
df_basecall = df_basecall[["nparts", "dir_output"]]
df_basecall['sample'] = df_basecall.index
df_basecall.index = df_basecall.index + df_basecall.nparts.astype(str)
df_basecall.dir_output = df_basecall.dir_output + df_basecall.nparts.astype(str) + '/'
df_basecall['basecalledDir'] = df_basecall.index + '/'
df_basecall['guppy'] = config['guppy']
df_basecall['model'] = config['model']
for column in ['basecalledDir']:
    df_basecall[column] = resultDir + 'step2_basecall/' + df_basecall[column]

def parseDfToInput_basecall_splitH5(wildcard):
    selfWildCardUnique = True
    if isinstance(df_basecall.at[wildcard.sampleSplit, 'sample'], list):
        selfWildCardUnique = False
    if selfWildCardUnique:
        return resultDir + 'step1_splitH5/' + df_basecall.at[wildcard.sampleSplit, 'sample'] + '.fi

In [11]:
df_mergeFq = df_basecall.reset_index().groupby("sample")["index"].agg(list).pipe(
    pd.DataFrame
).rename(columns={"index": "sampleSplit"}).assign(dir_out=lambda df: df.index + "/")

In [13]:
rule_mergeFq = jps.SnakeRule(sf, 'mergeFq', 18, 0)
rule_mergeFq.addCode("""
df_mergeFq = df_basecall.reset_index().groupby("sample")["index"].agg(list).pipe(
    pd.DataFrame
).rename(columns={"index": "sampleSplit"}).assign(dir_out=lambda df: df.index + "/")
""")
rule_mergeFq.addMetaDf('df_mergeFq', ['dir_out'], metaDf=df_mergeFq)
rule_mergeFq.addMain('params', ['basecalledDir'], fromRule=rule_basecall)
rule_mergeFq.addMain('params', ['dir_out'])
rule_mergeFq.setShell("""
mkdir -p {params.dir_out}
cd {pipelineDir}
python mergeFastq.py {params.basecalledDir} -o {params.dir_out} -t {threads}
""")
rule_mergeFq

2021-09-27 15:18:54.740 | INFO     | jpy_tools.parseSnake2:addRule:55 - mergeFq step num: 3



## get parameter of rule `mergeFq` ##
df_mergeFq = df_basecall.reset_index().groupby("sample")["index"].agg(list).pipe(
    pd.DataFrame
).rename(columns={"index": "sampleSplit"}).assign(dir_out=lambda df: df.index + "/")
for column in ['dir_out']:
    df_mergeFq[column] = resultDir + 'step3_mergeFq/' + df_mergeFq[column]

def parseDfToInput_mergeFq_basecall(wildcard):
    selfWildCardUnique = True
    if isinstance(df_mergeFq.at[wildcard.sample, 'sampleSplit'], list):
        selfWildCardUnique = False
    if selfWildCardUnique:
        return resultDir + 'step2_basecall/' + df_mergeFq.at[wildcard.sample, 'sampleSplit'] + '.finished'
    else:
        return [resultDir + 'step2_basecall/' + x + '.finished' for x in df_mergeFq.loc[wildcard.sample, 'sampleSplit']]

def parseDfToParams_mergeFq_basecall_basecalledDir(wildcard):
    selfWildCardUnique = True
    if isinstance(df_mergeFq.at[wildcard.sample, 'sampleSplit'], list):
        selfWildCardUnique = False
    if selfWildCardUnique

In [14]:
rule_all = jps.SnakeAll(sf, rule_mergeFq)
rule_all

rule all:
    input:
        mergeFqFinished = [resultDir + 'step3_mergeFq/' + "" + sample + ".finished" for sample in df_mergeFq.index],

In [15]:
sf.getMain(path_sf)

import pandas as pd
#configfile: "/public/home/liuzj/scripts/pipeline/basecallByGuppy/snakemake/config.yaml"
pipelineDir = config['pipelineDir']
resultDir = config["resultDir"].rstrip("/") + "/"
pipelineDir = config["pipelineDir"].rstrip("/") + "/"


## get parameter of rule `splitH5` ##
import pandas as pd
dt_h5 = config['input']
df_splitH5 = pd.DataFrame.from_dict(dt_h5).T
df_splitH5['dir_output'] = df_splitH5.index + '/'
for column in ['dir_output']:
    df_splitH5[column] = resultDir + 'step1_splitH5/' + df_splitH5[column]


## get parameter of rule `basecall` ##
df_basecall = df_splitH5.copy()
df_basecall["nparts"] = df_basecall["nparts"].map(lambda x: list(range(x)))
df_basecall = df_basecall.explode("nparts")
df_basecall = df_basecall[["nparts", "dir_output"]]
df_basecall['sample'] = df_basecall.index
df_basecall.index = df_basecall.index + df_basecall.nparts.astype(str)
df_basecall.dir_output = df_basecall.dir_output + df_basecall.nparts.astype(str) + '/'
df_basecall['basecalle